## GTSEP v0

### Description
Model for specialization project. Binary decision variables for generation. Capped battery investments. No ramping constraints. 

### Indexes and index sets

- $n \in N$: Set of nodes.
- $i \in G^{old}$: Set of existing generators. Each generator is associated with a node $n$.
- $i \in G^{new}$: Set of new generators. Each generator is associated with a node $n$.
- $i \in G$: Set of all generators.
- $i \in G_n$: Set of generators at node $n$, including new generators.
- $b \in B^{new}$: Set of new branches.
- $b \in B^{old}$: Set of existing branches.
- $b \in B$: Set all of branches.
- $b \in B_n^{in}$: Set of branches coming into node $n$, including new branches.
- $b \in B_n^{out}$: Set of branches going out of node $n$, including new branches.
- $s \in S^{old}$: Set of new batteries. Each battery is associated with a node $n$.
- $s \in S^{new}$: Set of new batteries. Each battery is associated with a node $n$.
- $s \in S$: Set of all batteries. Each battery is associated with a node $n$.
- $s \in S_n$: Set of batteries at node $n$, including new batteries.
- $t \in T$: Set of time periods.

### Parameters

- $P_{i}^{\min}$: Minimum power output of generator $i$ (MW)
- $P_{i}^{\max}$: Maximum power output of generator $i$ (MW)
- ${VOLL}$: Value of lost load (cost of load shedding) ($/MWh)
- $CC$: Cost of curtailment (\$/MWh)
- $MC_{i}$: Marginal cost of generator $i$ (\$/MWh)
- $CO2_{i}$ Cost of CO2 emissions of generator $i$ (\$/MWh)
- $E_{i}$: CO2 emissions of generator $i$ (ton/MWh)
- $E_{limit}$: CO2 emissions limit (ton)
- $D_{n,t}$: Demand at node $n$ and time $t$ (MW)
- $l_b$: loss factor of branch $b$ (given, but in reality some function of distance, transmsission line type, etc.)
- $P_{b}^{\max}$: Maximum power flow on branch $b$ (MW)
- $\eta_{s}^{ch}$: charge efficiency of battery s
- $\eta_{s}^{dis}$: discharge efficiency of battery s
- $P_{s}^{ch, \max}$: Maximum charging power of battery $s$ (MW)
- $P_{s}^{ch, \min}$: Minimum charging power of battery $s$ (MW)
- $P_{s}^{dis, \max}$: Maximum discharge power output of battery $s$ (MW)
- $P_{s}^{dis, \min}$: Minimum discharge power output battery $s$ (MW)
- $SOC_{s}^{\max}$: Maximum state of charge of battery $s$ (MWh)
- $SOC_{s}^{\min}$: Minimum state of charge of battery $s$ (MWh)
- $MC_{s}^{dis}$: Marginal cost of discharging battery $s$ (\$/MWh)
- $cf_{i, t}$: Capacity factor of generator $i$ at time $t$. Equals 1 for all non-renewable generators, otherwise $\in (0, 1)$.
- $P_b^{min}$: Minimum capacity of new branch $b$ (MW)
- $P_{b}^{\max}$: Maximum power flow on branch $b$ (MW)
- $AIC_{i}$: Annualized investment cost of new generator $i$ (\$/MW)
- $AIC_{s}$: Annualized investment cost of new battery $s$ (\$/MWh)
- $AIC_{b}$: Annualized investment cost of new branch $b$ (\$/MW)

### Decision variables

- $g_{i,t}$: Power generation dispatch of generator $i$ at time $t$ (MW)
- $f_{b,t}$: Power flow on branch $b$ at time $t$ (MW)
- $sh_{n,t}$: Load shedding at node $n$ at time $t$ (MW)
- $c_{i,t}$: Power curtailment at generator $i$ at time $t$ (MW)
- $g^{ch}_{s,t}$: Charging power of battery $s$ at time $t$ (MW)
- $g^{dis}_{s,t}$: Discharging power of battery $s$ at time $t$ (MW)
- $soc_{s,t}$: State of charge of storage unit $s$ at time $t$ (MWh)
- $x_{i}$: Binary variable that indicates if generator $i$ is built or not (only exists for new generators)
- $y_{b}$: Binary variable that indicates if branch $b$ is built or not (only exists for new branches)
- $z_{s}$: Binary variable that indicates if battery $s$ is built or not (only exists for new batteries)
- $p_i^{max}$: Maximum capacity of new generator i (MW)
- $p_b^{max}$: Maximum capacity of new branch b (MW)

### Optimization Model

### Objective function, minimize cost of generation

**Minimize:**
$$ 
\sum_{i \in G} \sum_{t \in T} \underbrace{(MC_{i} + CO2_{i}) g_{i,t}}_{\text{Generation cost}} 
+ \sum_{s \in S} \sum_{t \in  T} \underbrace{MC_{s} g_{s,t}^{dis} \eta_{s}^{dis}}_{\text{Battery discharge cost}}
+ \sum_{n \in N} \sum_{t \in T} \underbrace{sh_{n,t} VOLL}_{\text{Load shedding cost}} 
+ \sum_{n \in N} \sum_{t \in T} \underbrace{c_{n,t} CC}_{\text{Curtailment cost}} 
+ \underbrace{AIC}_{\text{Annualized investment cost}}
$$

where

$$ 
AIC = \underbrace{\sum_{i \in G^{new}} AIC_{i} p_i^{max}}_{\text{Generator expansion cost}} 
+ \underbrace{\sum_{b \in B^{new}} AIC_{b} p_b^{max}}_{\text{Transmission expansion cost}} 
+ \underbrace{\sum_{s \in S^{new}} AIC_{s} SOC_{s}^{max} z_{s}}_{\text{Storage expansion cost}} 
$$


1. **Power balance: production + inflow - curtailment = outflow + demand - shedding**

_A.K.A. Market clearing or energy balance_

$$ \sum_{i \in G_n} (g_{i,t} - c_{i,t}) + \sum_{b \in B_n^{in}} f_{b,t} (1 - l_{b}) - \sum_{b \in B_n^{out}} f_{b,t} - \sum_{s \in S_n} (g_{s,t}^{ch} - \eta_{s}^{dis} g_{s,t}^{dis}) + sh_{n,t} = D_{n,t} \quad \forall n \in N, \forall t \in T $$

2. **a) We can't shed more load than the demand**

$$ sh_{n,t} \leq D_{n,t} \quad \forall n \in N, \forall t \in T $$

2. **b) We can't curtail more energy than is produced**

$$ 0 \leq c_{i,t} \leq g_{i,t} \quad \forall i \in G, \forall t \in T $$

3. **a) Generators' power output limits, old generators**

$$ P_{i}^{\min} \leq g_{i,t} \leq P_{i}^{\max} {cf}_{i,t} \quad \forall i \in G^{old}, \forall t \in T $$

where ${cf}_{i,t}$ is 1 $ \forall t$ if the generator is fossil, but varies if the generator is renewable.

3. **b) Generators' power output limits, new generators**

$$ x_{i} P_{i}^{\min} \leq g_{i,t} \leq x_i p_{i}^{\max} {cf}_{i,t}  \quad \forall i \in G^{new}, \forall t \in T $$

where ${cf}_{i,t}$ is 1 $\forall t$ if the generator is fossil, but varies if the generator is renewable.

3. **c) New generators' isntalled capacity limit**

$$ p_i^{max} \leq P_i^{\max} \quad \forall i \in G^{new} $$

4. **a) Branch power flow limits, old branches**

$$ -P_{b}^{\max} \leq f_{b,t} \leq P_{b}^{\max} \quad \forall b \in B^{old}, \forall t \in T $$

4. **b) Branch power flow limits, new branches**

$$ -y_b p_{b}^{\max} \leq f_{b,t} \leq y_b p_{b}^{\max} \quad \forall b \in B^{new}, \forall t \in T $$

4. **c) Branch max_capacity limits, new branches**

$$ y_b P_b^{min} \leq p_{b}^{\max} \leq y_b P_{b}^{\max} \quad \forall b \in B^{new} $$

5. **Emissions restrictions**

$$ \sum_{i \in G} \sum_{t \in T} E_{i} g_{i,t} \leq E_{limit} $$

6. **a) Battery charging limit, old batteries**

$$ P_{s}^{ch, \min} \leq g_{s,t}^{ch} \leq P_{s}^{ch, \max} \quad \forall s \in S^{old}, \forall t \in T $$

6. **b) Battery charging limit, new batteries**

$$ z_s P_{s}^{ch, \min} \leq g_{s,t}^{ch} \leq z_s P_{s}^{ch, \max} \quad \forall s \in S^{new}, \forall t \in T $$

7. **a) Battery discharging limit, old batteries**

$$ P_{s}^{dis, \min} \leq g_{s,t}^{dis} \leq P_{s}^{dis, \max} \quad \forall s \in S^{old}, \forall t \in T $$

7. **b) Battery discharging limit, new batteries**

$$ z_s P_{s}^{dis, \min} \leq g_{s,t}^{dis} \leq z_s P_{s}^{dis, \max} \quad \forall s \in S^{new}, \forall t \in T $$

8. **Battery state of charge limits**

$$ SOC_{s}^{\min} \leq soc_{s,t} \leq SOC_{s}^{\max} \quad \forall s \in S, \forall t \in T $$

9. **Battery state of charge dynamics**

$$ soc_{s,t} = soc_{s,t-1} + \eta_{s}^{ch} g_{s,t}^{ch} - \frac{1}{\eta_{s}^{dis}} g_{s,t}^{dis} \quad \forall s \in S, \forall t \in T - \{0\} $$

10. **Battery state of charge at time 0**

$$ soc_{s,0} = SOC_{s}^{\min} \quad \forall s \in S $$

11. **Variable definitions**

All continuous variables are non-negative:
$$ 
g_{i,t}, f_{b,t}, sh_{n,t}, c_{n,t}, g_{s,t}^{ch}, g_{s,t}^{dis}, soc_{s,t}, p_i^{max}, p_b^{max}, soc_s \geq 0, 
\quad \forall i \in G, \forall b \in B, \forall n \in N, \forall s \in S, \forall t \in T 
$$

All binary variables are restricted to 0 or 1:
$$ 
x_i, y_b, z_s \in \{0,1\}, 
\quad \forall i \in G^{new}, \forall b \in B^{new}, \forall s \in S^{new} 
$$